## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
# answer below:
wine.isnull().mean()

#no missing data

fixed.acidity           0.0
volatile.acidity        0.0
citric.acid             0.0
residual.sugar          0.0
chlorides               0.0
free.sulfur.dioxide     0.0
total.sulfur.dioxide    0.0
density                 0.0
pH                      0.0
sulphates               0.0
alcohol                 0.0
quality                 0.0
dtype: float64

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [4]:
# answer below
wine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [5]:
#making new column for quality seperating at the 5 and 6 line
#reviews['New Rating'] = reviews['Rating'].apply(lambda x: 1 if x > 4.5 else 0)
wine['high_quality'] = wine['quality'].apply(lambda x: 1 if x > 5 else 0)

In [6]:
#check
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,high_quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


Create a train test split with 20% of the data in the test subsample.

In [7]:
# answer below:
from sklearn.model_selection import train_test_split

X = wine.drop(['quality', 'high_quality'], axis=1)
y = wine['high_quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Scale only the independent variables using the minmax scaler.

In [8]:
# answer below:
from sklearn.preprocessing import MinMaxScaler

scale = MinMaxScaler()

X_train_scale = scale.fit_transform(X_train)
X_test_scale = scale.transform(X_test)

Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [9]:
# answer below:
from sklearn.neighbors import KNeighborsClassifier

#regular X_train, X_test accuracy scores
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
train_score = knn.score(X_train, y_train)
test_score = knn.score(X_test, y_test)

#scaled X_train, X_test
knn.fit(X_train_scale, y_train)
train_scaled_score = knn.score(X_train_scale, y_train)
test_scaled_score = knn.score(X_test_scale, y_test)

print('Before scaling')
print('Train accuracy score: {}\nTest accuracy score {}'.format(train_score, test_score))
print('\nAfter scaling')
print('Train accuracy score: {}\nTest accuracy score {}'.format(train_scaled_score, test_scaled_score))

Before scaling
Train accuracy score: 0.7693510555121188
Test accuracy score 0.6125

After scaling
Train accuracy score: 0.8209538702111024
Test accuracy score 0.740625


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [24]:
#Write a custom weight function that assigns the weight of 1/sqrt(distance)
def _weight_inv_sqrt(distance):
  weights = []
  with np.errstate(divide='ignore'):
    distance = 1. / distance
  inf_mask = np.isinf(distance)
  inf_row = np.any(inf_mask, axis=1)
  distance[inf_row] = inf_mask[inf_row]
  return distance

In [25]:
knn2 = KNeighborsClassifier(n_neighbors=5, weights=_weight_inv_sqrt)
knn2.fit(X_train_scale, y_train)

train_weighted_score = knn2.score(X_train_scale, y_train)
test_weighted_score = knn2.score(X_test_scale, y_test)

print('Weighted scores')
print('Train score: {}\nTest score: {}'.format(train_weighted_score, test_weighted_score))

Weighted scores
Train score: 1.0
Test score: 0.80625


In [26]:
#comparing all scores
print('Original')
print('Train accuracy score: {}\nTest accuracy score {}'.format(train_score, test_score))
print('\nAfter scaling')
print('Train accuracy score: {}\nTest accuracy score {}'.format(train_scaled_score, test_scaled_score))
print('\nAfter weight function')
print('Train score: {}\nTest score: {}'.format(train_weighted_score, test_weighted_score))

Original
Train accuracy score: 0.7693510555121188
Test accuracy score 0.6125

After scaling
Train accuracy score: 0.8209538702111024
Test accuracy score 0.740625

After weight function
Train score: 1.0
Test score: 0.80625


In [20]:
#Write a custom weight function that assigns the weight of 1/sqrt(distance)
#def _weight_inv_sqrt(distance):
#  weights = []
#  for d in range(len(distance)):
#    if distance[d] != 0:
#      weights.append([1 / np.sqrt(distance[d][c])])
#    else:
#      np.isinf
#  return distance

In [ ]:
# answer below:
#sklearn function
'''
def _get_weights(dist, weights):
    """Get the weights from an array of distances and a parameter ``weights``
    Parameters
    ----------
    dist : ndarray
        The input distances
    weights : {'uniform', 'distance' or a callable}
        The kind of weighting used
    Returns
    -------
    weights_arr : array of the same shape as ``dist``
        if ``weights == 'uniform'``, then returns None
    """
    if weights in (None, 'uniform'):
        return None
    elif weights == 'distance':
        # if user attempts to classify a point that was zero distance from one
        # or more training points, those training points are weighted as 1.0
        # and the other points as 0.0
        if dist.dtype is np.dtype(object):
            for point_dist_i, point_dist in enumerate(dist):
                # check if point_dist is iterable
                # (ex: RadiusNeighborClassifier.predict may set an element of
                # dist to 1e-6 to represent an 'outlier')
                if hasattr(point_dist, '__contains__') and 0. in point_dist:
                    dist[point_dist_i] = point_dist == 0.
                else:
                    dist[point_dist_i] = 1. / point_dist
        else:
            with np.errstate(divide='ignore'):
                dist = 1. / dist
            inf_mask = np.isinf(dist)
            inf_row = np.any(inf_mask, axis=1)
            dist[inf_row] = inf_mask[inf_row]
        return dist
    elif callable(weights):
        return weights(dist)
    else:
        raise ValueError("weights not recognized: should be 'uniform', "
                         "'distance', or a callable function")
'''

In [ ]:
#import numpy as np
#def isd():
#    w= []
#    for i in range(len(x_train)):
        # first we compute the euclidean distance
#        distance = np.sqrt(np.sum(np.square(x_test 
#        x_train[i,:])))
        # add it to list of distances
#        w.append([1/(np.square(distance)), i]) 
#        return w